In [1]:
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


# GPU로 실행할 것!!

In [5]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

# 이미지 보여주는 함수

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

# 보간법을 통해서 이미지를 보여준다.     

In [22]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [8]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

model_LeNet5 = keras.models.Sequential([
    DefaultConv2D(filters=6, kernel_size=5, input_shape=[28, 28, 1]),
    keras.layers.AveragePooling2D(pool_size=2, strides=2),
    DefaultConv2D(filters=16, kernel_size=5),
    keras.layers.AveragePooling2D(pool_size=2, strides=2),
    DefaultConv2D(filters=120),
    keras.layers.Flatten(),
    keras.layers.Dense(units=84, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

In [23]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='tanh', padding="SAME")

model_LeNet5_tanh = keras.models.Sequential([
    DefaultConv2D(filters=6, kernel_size=5, input_shape=[28, 28, 1]),
    keras.layers.AveragePooling2D(pool_size=2, strides=2),
    DefaultConv2D(filters=16, kernel_size=5),
    keras.layers.AveragePooling2D(pool_size=2, strides=2),
    DefaultConv2D(filters=120),
    keras.layers.Flatten(),
    keras.layers.Dense(units=84, activation='tanh'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

In [24]:
model_LeNet5.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model_LeNet5.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
score = model_LeNet5.evaluate(X_test, y_test)
X_new = X_test[:10] # pretend we have new images
y_pred = model_LeNet5.predict(X_new)

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.1796 - accuracy: 0.9319 - val_loss: 0.3007 - val_accuracy: 0.9054
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.1682 - accuracy: 0.9365 - val_loss: 0.3056 - val_accuracy: 0.9058
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.1554 - accuracy: 0.9417 - val_loss: 0.3105 - val_accuracy: 0.9102
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.1438 - accuracy: 0.9440 - val_loss: 0.2946 - val_accuracy: 0.9088
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.1370 - accuracy: 0.9479 - val_loss: 0.2898 - val_accuracy: 0.9126
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.1323 - accuracy: 0.9501 - val_loss: 0.3640 - val_accuracy: 0.9048
Epoch 7/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.1254 - accuracy: 0.9523 - val_loss: 0.3400 - val_accuracy:

In [25]:
model_LeNet5_tanh.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model_LeNet5_tanh.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
score = model_LeNet5_tanh.evaluate(X_test, y_test)
X_new = X_test[:10] # pretend we have new images
y_pred = model_LeNet5_tanh.predict(X_new)

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5582 - accuracy: 0.8011 - val_loss: 0.3460 - val_accuracy: 0.8742
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3746 - accuracy: 0.8683 - val_loss: 0.2936 - val_accuracy: 0.8898
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3256 - accuracy: 0.8836 - val_loss: 0.2714 - val_accuracy: 0.9018
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2902 - accuracy: 0.8956 - val_loss: 0.2606 - val_accuracy: 0.9046
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2660 - accuracy: 0.9023 - val_loss: 0.2429 - val_accuracy: 0.9096
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2468 - accuracy: 0.9111 - val_loss: 0.2541 - val_accuracy: 0.9084
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2297 - accuracy: 0.9152 - val_loss: 0.2496 - val_accuracy: